# **Import libraries**

In [ ]:

"""Import necessary libraries"""
import numpy as np
import tensorflow
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import re
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, preprocessing



# **Import Dataset**

In [ ]:
dataset=pd.read_csv("superheroes_nlp_dataset.csv")
dataset

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,50,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,Zatanna,Zatanna Zatara,Zatanna Zatara,10,Zatanna is the daughter of adventurer John Zat...,Zatanna is genetically talented with her magi...,90,10,25,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1446,Zero,DWN-∞: Zero,DWN-∞: Zero,18,Zero was created by the late Dr. Albert Wily ...,NaN,80,100,100,100,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1447,Zoom (New 52),Hunter Zolomon,NaN,20,"Hunter Zolomon is better known as Zoom, a spee...",After tricking Barry Allen and Wally West into...,95,50,100,75,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1448,Zoom,Hunter Zolomon,Hunter Zolomon,9,Hunter Zolomon had a troubled relationship wi...,"Zoom is able to alter time, to make himself ev...",75,10,100,30,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
# classify_dataset=dataset[['history_text','powers_text','creator']]
classify_dataset=dataset[['history_text','creator']]
classify_dataset

,history_text,creator
0,"Delroy Garrett, Jr. grew up to become a track ...",Marvel Comics
1,He was one of the many prisoners of Indian Hil...,DC Comics
2,"Richard ""Rick"" Jones was orphaned at a young ...",Marvel Comics
3,Aa is one of the more passive members of the P...,DC Comics
4,Aaron Cash is the head of security at Arkham A...,DC Comics
...,...,...
1445,Zatanna is the daughter of adventurer John Zat...,DC Comics
1446,Zero was created by the late Dr. Albert Wily ...,Capcom
1447,"Hunter Zolomon is better known as Zoom, a spee...",DC Comics
1448,Hunter Zolomon had a troubled relationship wi...,DC Comics


In [ ]:
# classify_dataset["powers_text"].isna().sum()
classify_dataset["history_text"].isna().sum()
classify_dataset=classify_dataset.dropna(axis=0)

# **Data Preprocessing**

In [ ]:
import nltk 
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
"""Clean the data"""
def preprocess(text,stop_words,stemming=False,lemantizing=False):
  
  text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
  #tokenize
  text_list=text.split()
  # text_list=[str(word) for word in text_list]
  #stopwords
  if stop_words:
    text_list=[word for word in text_list if word not in stop_words]
  
  ## Stemming 
  if stemming ==True:
    st=nltk.stem.porter.PorterStemmer()
    text_list=[st.stem(word) for word in text_list]
    
  ## Lemmatisation 
  if lemantizing==True:
    le=nltk.stem.wordnet.WordNetLemmatizer()
    text_list=[le.lemmatize(word) for word in text_list]

   ## tokenize to string again
    text = " ".join(text_list)
    return text



In [ ]:
#Clean the history_text

stopwords_list=nltk.corpus.stopwords.words("english")

classify_dataset["clean_history"]=classify_dataset["history_text"].apply(lambda x: preprocess(x,stop_words=stopwords_list,stemming=False,lemantizing=True))
classify_dataset.head


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


<bound method NDFrame.head of                                            history_text        creator  \
0     Delroy Garrett, Jr. grew up to become a track ...  Marvel Comics   
1     He was one of the many prisoners of Indian Hil...      DC Comics   
2      Richard "Rick" Jones was orphaned at a young ...  Marvel Comics   
3     Aa is one of the more passive members of the P...      DC Comics   
4     Aaron Cash is the head of security at Arkham A...      DC Comics   
...                                                 ...            ...   
1445  Zatanna is the daughter of adventurer John Zat...      DC Comics   
1446   Zero was created by the late Dr. Albert Wily ...         Capcom   
1447  Hunter Zolomon is better known as Zoom, a spee...      DC Comics   
1448   Hunter Zolomon had a troubled relationship wi...      DC Comics   
1449  Zzzax was the product of an accident that occu...  Marvel Comics   

                                          clean_history  
0     delroy garrett jr

# **Split Data**

In [ ]:
"""Split the data"""
from sklearn.model_selection import train_test_split

# creating instance of labelencoder
labelencoder = preprocessing.LabelEncoder()
classify_dataset['creator_labels'] = labelencoder.fit_transform(classify_dataset['creator'])
X_train,X_test,Y_train,Y_test=train_test_split(classify_dataset['clean_history'],classify_dataset['creator_labels'],test_size=0.3)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
Y_train.value_counts()

# **bag-of-words**

In [ ]:
#vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(max_features=10000,ngram_range=(1,2),lowercase=False)


In [ ]:
#ML model

from sklearn.svm import SVC
classifier= SVC(kernel='linear',random_state=42)


In [ ]:
model=pipeline.Pipeline([("vectorizer",vectorizer),
                         ("classifier",classifier)])

model.fit(X_train,Y_train)
#Test
# predicted=model.predict(X_test)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(lowercase=False, max_features=10000,
                                 ngram_range=(1, 2))),
                ('classifier', SVC(kernel='linear', random_state=42))])

In [ ]:
predicted=model.predict(X_test)

In [ ]:
# Accuracy, Precision, Recall
from sklearn import metrics
train_accuracy = metrics.accuracy_score(Y_train, model.predict(X_train))
test_accuracy = metrics.accuracy_score(Y_test, predicted)
print("Train Accuracy:",  round(train_accuracy,2))
print("Test Accuracy:",  round(test_accuracy,2))
print("Detail:")
print(metrics.classification_report(Y_test, predicted))

Train Accuracy: 0.96
Test Accuracy: 0.82
Detail:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           5       0.82      0.88      0.85       122
           6       1.00      0.50      0.67         8
           7       0.00      0.00      0.00         1
           9       1.00      0.40      0.57         5
          10       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         3
          13       0.00      0.00      0.00         1
          16       0.00      0.00      0.00         2
          17       1.00      0.25      0.40         4
          18       0.00      0.00      0.00         3
          19       0.00      0.00      0.00         1
          21       1.00      0.50      0.67         2
          22       0.81      0.97      0.88       187
          25       0.00      0.0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **Word2vec**

In [ ]:
"""Import libraries"""
import gensim
from gensim import models
import gensim.downloader as gensim_api
import tensorflow
from tensorflow.keras import models,layers,preprocessing 
from tensorflow.keras import backend as K

In [ ]:
"""Clean the data"""
doc=X_train

#creating unigrams
doc_list=[]
for sentence in doc:
   text_list=sentence.split()
   list_grams = [" ".join(text_list[i:i+1]) 
               for i in range(0, len(text_list), 1)]
   doc_list.append(list_grams)

## detect bigrams and trigrams
bigrams_detector = gensim.models.phrases.Phrases(doc_list, 
                 delimiter=" ".encode(), min_count=5, threshold=10)
bigrams_detector = gensim.models.phrases.Phraser(bigrams_detector)
trigrams_detector = gensim.models.phrases.Phrases(bigrams_detector[doc_list], 
            delimiter=" ".encode(), min_count=5, threshold=10)
trigrams_detector = gensim.models.phrases.Phraser(trigrams_detector)

In [ ]:
#fit w2v model
nlp=gensim.models.word2vec.Word2Vec(doc_list,size=300,window=8,min_count=1,sg=1,iter=30)

In [ ]:
# Save word2vec model
nlp.save("word2vec_superhero.model")

In [ ]:
#tokenizer
tokenizer=preprocessing.text.Tokenizer(lower=True, split= ' ',oov_token="Nan", filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(doc_list)
dictionary_vocab=tokenizer.word_index

text2seq=tokenizer.texts_to_sequences(doc_list)

#padding
X_train_wv=preprocessing.sequence.pad_sequences(text2seq,maxlen=15,padding='post',truncating="post")




In [ ]:
doc_test = X_test

## create list of n-grams'
doc_test_list = []
for string in doc_test:
    lst_words = string.split()
    lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, 
                 len(lst_words), 1)]
    doc_test_list.append(lst_grams)
    
## detect common bigrams and trigrams using the fitted detectors
lst_corpus = list(bigrams_detector[doc_test_list])
lst_corpus = list(trigrams_detector[lst_corpus])
## text to sequence with the fitted tokenizer
lst_text2seq = tokenizer.texts_to_sequences(lst_corpus)

## padding sequence
X_test_wv = preprocessing.sequence.pad_sequences(lst_text2seq, maxlen=15,
             padding="post", truncating="post")

In [ ]:
## start the matrix (length of vocabulary x vector size) with all 0s
embeddings = np.zeros((len(dictionary_vocab)+1, 300))
for word,idx in dictionary_vocab.items():
    ## update the row with vector
    try:
        embeddings[idx] =  nlp[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
"""Model creation"""
model = keras.Sequential()
model.add(layers.Embedding(input_dim=embeddings.shape[0],output_dim=embeddings.shape[1] , weights=[embeddings], input_length=15, trainable=False))
# model.add(layers.Flatten())
model.add(layers.Bidirectional(layers.LSTM(units=15, dropout=0.2)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(40, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 300)           7535100   
                                                                 
 bidirectional (Bidirectiona  (None, 30)               37920     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                1984      
                                                                 
 dense_1 (Dense)             (None, 40)                2600      
                                                                 
Total params: 7,577,604
Trainable params: 42,504
Non-trainable params: 7,535,100
_________________________________________________________________


In [ ]:
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
## encode y
dic_y_mapping = {n:label for n,label in 
                 enumerate(np.unique(Y_train))}
inverse_dic = {v:k for k,v in dic_y_mapping.items()}
Y_train = np.array([inverse_dic[y] for y in Y_train])
# ## train
pmodel=model.fit(x=X_train_wv, y=Y_train, batch_size=16, epochs=30, validation_split=0.1,callbacks=EarlyStopping(monitor='val_loss',verbose=1, patience=5))

Epoch 1/30
49/49 [==============================] - 6s 37ms/step - loss: 2.7754 - accuracy: 0.3866 - val_loss: 1.8239 - val_accuracy: 0.4828
Epoch 2/30
49/49 [==============================] - 1s 16ms/step - loss: 1.6584 - accuracy: 0.5103 - val_loss: 1.6679 - val_accuracy: 0.5977
Epoch 3/30
49/49 [==============================] - 1s 16ms/step - loss: 1.4674 - accuracy: 0.6430 - val_loss: 1.4697 - val_accuracy: 0.6437
Epoch 4/30
49/49 [==============================] - 1s 16ms/step - loss: 1.2438 - accuracy: 0.7075 - val_loss: 1.3916 - val_accuracy: 0.6437
Epoch 5/30
49/49 [==============================] - 1s 17ms/step - loss: 1.0713 - accuracy: 0.7229 - val_loss: 1.3254 - val_accuracy: 0.6667
Epoch 6/30
49/49 [==============================] - 1s 18ms/step - loss: 0.9459 - accuracy: 0.7564 - val_loss: 1.2782 - val_accuracy: 0.6782
Epoch 7/30
49/49 [==============================] - 1s 16ms/step - loss: 0.8452 - accuracy: 0.7784 - val_loss: 1.2771 - val_accuracy: 0.6667
Epoch 8/30
49

In [ ]:
# model.evaluate(X_test_wv, y_test_p)
from sklearn import metrics
predicted_prob = model.predict(X_test_wv)
predicted = [dic_y_mapping[np.argmax(pred)] for pred in 
             predicted_prob_p]
metrics.accuracy_score(Y_test,predicted)

12/12 [==============================] - 0s 11ms/step


0.6837837837837838